In [1]:
import pandas as pd 
import numpy as np 

In [2]:
country = 'mwi_mics19-20'

In [3]:
data = pd.read_csv('./'+ country)

In [4]:
data

,hh_id,psu,strata,weight,d_cm,d_nutr,d_satt,d_educ,d_elct,d_sani,...,cookingfuel,television,radio,telephone,refrigerator,car,bicycle,motorbike,animal_cart,computer
0,101,1,Chitipa Rural,0.432607,0.0,NaN,0.0,0.0,1.0,0.0,...,WOOD,No,No,No,No,No,No,No,No,No
1,102,1,Chitipa Rural,0.432607,0.0,1.0,0.0,1.0,1.0,1.0,...,WOOD,No,No,Yes,No,No,Yes,No,No,No
2,103,1,Chitipa Rural,0.432607,0.0,1.0,0.0,0.0,1.0,1.0,...,WOOD,No,No,Yes,No,No,No,No,No,No
3,104,1,Chitipa Rural,0.432607,0.0,0.0,0.0,0.0,1.0,0.0,...,WOOD,No,Yes,Yes,No,No,Yes,No,No,No
4,106,1,Chitipa Rural,0.432607,0.0,0.0,0.0,1.0,1.0,1.0,...,WOOD,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25414,111220,1112,Blantyre City,1.204571,0.0,0.0,0.0,1.0,1.0,1.0,...,WOOD,No,Yes,No,No,No,No,No,No,No
25415,111221,1112,Blantyre City,1.204571,0.0,1.0,0.0,0.0,1.0,1.0,...,WOOD,Yes,Yes,Yes,No,No,Yes,No,No,No
25416,111222,1112,Blantyre City,1.204571,0.0,0.0,1.0,0.0,1.0,1.0,...,CHARCOAL,Yes,Yes,Yes,Yes,No,No,Yes,No,No
25417,111223,1112,Blantyre City,1.204571,0.0,0.0,0.0,0.0,1.0,1.0,...,WOOD,No,No,No,No,No,No,No,No,No


In [5]:
psu_df = data.groupby('psu').agg({'weight' : 'max' ,'strata' : 'max', 'region' : 'max'}).reset_index()

In [7]:
psu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   psu     1111 non-null   int64  
 1   weight  1111 non-null   float64
 2   strata  1111 non-null   object 
 3   region  1111 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 34.8+ KB


In [9]:
# Convert variables to correct data type:
for col in data.columns:
    data[col] = data[col].astype('category')

for col in psu_df.columns:
    psu_df[col] = psu_df[col].astype('category')

In [10]:
cols_to_float = ['weight','eduyears']
for col in cols_to_float:
    data[col] = data[col].astype('Float32')
psu_df.weight = psu_df.weight.astype('float32')
    
cols_to_int = ['sex_Female','sex_Male',
                'agec7_0-4','agec7_5-9', 'agec7_10-14',
                'agec7_15-17', 'agec7_18-59', 'agec7_60+',
                'hhsize','child_eligible','child_mortality']
for col in cols_to_int:
    data[col] = data[col].astype('Int32')

In [11]:
trn_df = data.drop(columns=['strata','no_missing_edu',
    'attendance_not currently attending', 'attendance_currently attending',
    'no_missing_atten', 'timetowater', 'underweight_0.0', 'underweight_1.0',
    'stunting_0.0', 'stunting_1.0', 'wasting_0.0', 'wasting_1.0'], axis=1)

In [13]:
# Add the deprivations score as control for better MPI prediction
indicators = ['d_cm','d_nutr', 'd_satt','d_educ', 'd_elct', 'd_wtr', 'd_sani', 'd_hsg', 'd_ckfl', 'd_asst']
weights  = np.array([1/6, 1/6, 1/6, 1/6, 1/18, 1/18, 1/18, 1/18, 1/18, 1/18])

In [14]:
trn_df['score'] = round((trn_df.loc[:,indicators].astype(float) * weights).sum(axis=1, skipna=False),3).astype('category') 

In [15]:
from mostlyai.sdk import MostlyAI
mostly = MostlyAI(local=True)

Initializing Synthetic Data SDK 5.0.0 in LOCAL mode 🏠

Connected to ]8;id=594532;file:///home/igirela/mostlyai\/home/igirela/]8;;\]8;id=60266;file:///home/igirela/mostlyai\mostlyai]8;;\ with 189 GB RAM, 56 CPUs, 1x Quadro P2000 available

In [17]:
config = {
    "name": "Multi-table Synthetic Data Generation",
    "tables": [
        {
            "name": "household_survey",
            "data": trn_df,
            "primary_key": "hh_id",
            "foreign_keys": [
                {"column": "psu", "referenced_table": "psu", "is_context": True},
            ],
            'tabular_model_configuration': {
                'model': 'MOSTLY_AI/Large',
                'value_protection': False,
            }
        },
        {
            "name": "psu",
            "data": psu_df,
            "primary_key": "psu",  # Just a PK, no FK here
        },
    ],
}

In [ ]:
g = mostly.train(config=config)

Created generator 4ea77871-efb3-4a24-a159-e3b2bddd1034

Started generator training

Output()